# Finger Detection and Counting

## Imports

In [ ]:
# import cv2
import numpy as np

from sklearn.metrics import pairwise

## Global Variables

In [2]:
# This background will be a global variable that is updated through a few functions
background = None

accumulated_weight = 0.5

# Manually setting up the ROI for grabbing the hand
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

## Finding Average Background Value

In [3]:
#This function calculates the weighted sum of the input image 
def calc_accum_avg(frame,accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    # computing weighted average
    cv2.accumulateWeighted(frame,background,accumulated_weight)

## Segmenting the Hand Region in Frame

In [4]:
def segment(frame,threshold_min=25):
    
    # Calculates the Absolute Differentce between the backgroud and the passed in frame
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    # Applying a threshold to the image to grab the foreground
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    # Grabbing the external contours form the image
    image,contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    else:
        # ASSUMING THE LARGEST EXTERNAL CONTOUR IN ROI, IS THE HAND
        hand_segment = max(contours,key=cv2.contourArea)
        
        # Returning both the hand segment and the thresholded hand image
        return (thresholded,hand_segment)

## Counting Fingers with a Convex Hull

In [5]:
def count_fingers(thresholded,hand_segment):
    
    # Calculating the convex hull of the hand segment
    conv_hull =cv2.convexHull(hand_segment)
    
    # the convex hull will have at least 4 most outward points, on the top, bottom, left, and right.
    # Grabbing those points by using argmin and argmax
    # And understanding the general array shape returned by the conv hull.

    # Finding the top, bottom, left , and right.
    # making sure they are in a tuple format
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    # the center of the hand is half way between the top and bottom and halfway between left and right
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    # finding the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    # Calculating the Euclidean Distance between the center of the hand and the left, right, top, and bottom.
    distance = pairwise.euclidean_distances([cX,cY],Y=[left,right,top,bottom])[0]
    
    # Grabbing the largest distance
    max_distance = distance.max()
    
    # Creating a circle with 90% radius of the max euclidean distance
    radius = int(0.9*max_distance)
    circumfrence = (2*np.pi*radius)

    # grabbing an ROI of only that circle
    circular_roi = np.zeros(thresholded[:2],dtype='uint8')
    
    # drawing the circular ROI
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    # Using bit-wise AND with the cirle ROI as a mask.
    # This then returns the cut out obtained using the mask on the thresholded hand image.
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    # Grabbing contours in circle ROI
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    # Finger count starts at 0
    count = 0
    
    # looping through the contours to see any more fingers.
    for cnt in contours:
        
        # Bounding box of countour
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        # Incrementing count of fingers based on two conditions:
        # 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        
        # 2. Number of points along the contour does not exceed 25% of the 
        #circumference of the circular ROI (otherwise it will count points off the hand)
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

## Running the program

In [ ]:
cam = cv2.VideoCapture(0)

# Intializing a frame count
num_frames = 0

while True:
    
    # getting the current frame
    ret, frame = cam.read()
    
    # flipping the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
     
    # cloneing the frame    
    frame_copy = frame.copy()
    
    # Grabbing the ROI from the frame
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    # Applying grayscale and blur to ROI
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    # For the first 30 frames I calculate the average of the background.
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy,'WAIT. GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
    else:
        
        # segmenting the hand region
        hand = segment(gray)
        
        # Checking if we were able to actually detect a hand
        if hand is not None:
            
            thresholded , hand_segment = hand
            
            # DRAWS CONTOURS AROUND REAL HAND IN LIVE STREAM
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            
            # Counting the fingers
            fingers = count_fingers(thresholded,hand_segment)
            
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Thresholded',thresholded)
    
    # Drawing ROI Rectangle on frame copy
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    # incrementing the number of frames for tracking
    num_frames += 1
    
    # Displaying the frame with segmented hand
    cv2.imshow('Finger Count',frame_copy)
    
    # Closing windows with Esc
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break

# Releasing the camera and destroying all the windows
cam.release()
cv2.destroyAllWindows()
    